<a href="https://colab.research.google.com/github/bahl-lab-konstanz/animal_physiology_class/blob/main/drosophila_larva_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import numpy as np
from skimage import draw
import pylab as pl
import imageio
from IPython.display import HTML
from base64 import b64encode

In [ ]:
n_larvae = 25


p_turn_if_odor_gets_stronger = 0.5
p_turn_if_odor_gets_weaker = 0.5

speed_if_odor_gets_stronger = 1
speed_if_odor_gets_weaker = 1

turn_angle_if_odor_get_stronger = 90
turn_angle_if_odor_get_weaker = 120



x_positions = np.random.random(n_larvae)*512
y_positions = np.random.random(n_larvae)*512
orientations = np.random.random(n_larvae)*360

writer = imageio.get_writer("model_larvae.mp4", fps=60, ffmpeg_params=["-b:v", "4M"])

previous_odor_concentration = np.zeros(n_larvae)

for i in range(1500):
    img = np.zeros((512, 512))

    for j in range(n_larvae):

        # Make a turn or go streight?

        odor_distance = np.sqrt((256-x_positions[j])**2 + (50-y_positions[j])**2)
        odor_concentration = 1 / (odor_distance + 0.1)

        # is it getting better or worse?
        if odor_concentration > previous_odor_concentration[j]:
            p_turn = p_turn_if_odor_gets_stronger
            speed = speed_if_odor_gets_stronger
            turn_angle = turn_angle_if_odor_get_stronger
        else:
            p_turn = p_turn_if_odor_gets_weaker
            speed = speed_if_odor_gets_weaker
            turn_angle = turn_angle_if_odor_get_weaker

        if np.random.random() < p_turn:
            angle_change = np.random.randn() * turn_angle
        else:
            angle_change = 0

        orientations[j] += angle_change
        x_positions[j] += np.cos(orientations[j] * np.pi / 180) * speed
        y_positions[j] += np.sin(orientations[j] * np.pi / 180) * speed
        previous_odor_concentration[j] = odor_concentration

        if x_positions[j] < 0:
            x_positions[j] = 0
            orientations[j] = np.random.random()*360

        if y_positions[j] < 0:
            y_positions[j] = 0
            orientations[j] = np.random.random()*360

        if x_positions[j] > 511:
            x_positions[j] = 511
            orientations[j] = np.random.random()*360

        if y_positions[j] > 511:
            y_positions[j] = 511
            orientations[j] = np.random.random()*360

        rr, cc = draw.circle(int(y_positions[j]), int(x_positions[j]), radius=5, shape=img.shape)
        img[rr, cc] = 255

        rr, cc = draw.circle_perimeter(50, 256, radius=10, shape=img.shape)
        img[rr, cc] = 128

    writer.append_data(img.astype(np.uint8))

writer.close()

In [ ]:
mp4 = open('model_larvae.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""<video width=400 controls><source src="%s" type="video/mp4"></video>""" % data_url)